In [1]:
import numpy as np
import xroms
from glob import glob
import matplotlib.pyplot as plt
import gc
import xarray as xr
import pandas as pd
import xrft as xrft
import warnings
import sys
import xrft
# import seaborn as sns
sys.path.append("/homes/metogra/iufarias/FeedbackSubmeso/useful/")
import romspickle

from dask.diagnostics import ProgressBar
import dask
# dask.config.set(**{'array.slicing.split_large_chunks': True})
from dask.distributed import Client, LocalCluster

import scipy.integrate as integ
warnings.filterwarnings("ignore")




/homes/metogra/iufarias/xroms/xroms/interp.py:18: UserWarning: xESMF is not installed, so `interpll` will not run.
  warnings.warn("xESMF is not installed, so `interpll` will not run.")
/homes/metogra/iufarias/miniconda3/envs/XROMS/lib/python3.8/site-packages/pyproj/__init__.py:90: UserWarning: pyproj unable to set database path.
  _pyproj_global_context_initialize()


In [5]:
client = Client(n_workers=90)
# client = Client(cluster)

In [4]:
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 90
Total threads: 180,Total memory: 503.40 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:35942,Workers: 90
Dashboard: http://127.0.0.1:8787/status,Total threads: 180
Started: Just now,Total memory: 503.40 GiB
Comm: tcp://127.0.0.1:35655,Total threads: 2
Dashboard: http://127.0.0.1:37511/status,Memory: 5.59 GiB
Nanny: tcp://127.0.0.1:39366,


## Reading CROCO (3D)

In [7]:
# chunks={'time_counter':10, 'x_rho':50, 'y_rho':190,'s_rho':80,'s_w':80}
chunks={'time_counter':1,'x_rho':1000,"x_u":1000,"x_v":1000,
                        'y_rho':1520,"y_u":1520,"y_v":1520,'s_rho':80,'s_w':80}


path='/data/pacific/lrenault/SASI/CROCO/'

# varf=xr.open_mfdataset(path + 'FULL/SASI*2012-01*.nc',data_vars='minimal',chunks=chunks)
# vart=xr.open_mfdataset(path + 'SMTH/SASI*2012-01*.nc',data_vars='minimal',chunks=chunks)

varf=xr.open_mfdataset(path + 'FULL/SASI*2012*.nc',data_vars='minimal',chunks=chunks)
vart=xr.open_mfdataset(path + 'SMTH/SASI*2012*.nc',data_vars='minimal',chunks=chunks)



In [14]:
if varf.time_counter.shape[0]>150:
    month='total'
else:
    month='{0:02d}'.format(pd.DatetimeIndex(varf.time_counter)[0].month)
print(month)

total


Converting to xroms

In [13]:
varx = varf.rename({"time_counter":"ocean_time","x_rho": "xi_rho","x_u":"xi_u","x_v":"xi_v","x_w":"xi_w",
                    "y_rho":"eta_rho","y_v":"eta_v","y_u":"eta_u","y_w":"eta_w","s_rho":"s_rho","s_w":"s_w"})
df,gf=xroms.roms_dataset(varx,Vtransform=varx.Vtransform.data)

vary = vart.rename({"time_counter":"ocean_time","x_rho": "xi_rho","x_u":"xi_u","x_v":"xi_v",
                    "y_rho":"eta_rho","y_v":"eta_v","y_u":"eta_u","y_w":"eta_w","s_rho":"s_rho","s_w":"s_w"})
dt,gt=xroms.roms_dataset(vary,Vtransform=vary.Vtransform.data)

In [15]:

plt.rcParams['text.usetex'] = True
plt.rcParams.update({'font.size': 22})

plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.sans-serif'] = ['Lucida Grande']

In [16]:
timer_h=pd.DatetimeIndex(dt.ocean_time)
timer_d=pd.DatetimeIndex(dt.ocean_time.groupby("ocean_time.dayofyear").mean())

#### Spectral parameters

In [17]:
dx=0.5
nf=2
wdws='hann' #'flattop','hann'
wdw_cor=True
scl='density'

In [18]:
z_interp=np.linspace(-5,-50, 10)
fnum=240

### $N^2_r$
JAN=2.275e-06
FEB=4.178e-06
MAR=2.116e-05
APR=2.863e-05
MAY=1.790e-05
JUN=5.648e-05
AVG=2.175e-05

In [13]:
# %%time

# N2=[]
# for ixx in range(rho_full.shape[0]):
#     N2.append(df.xroms.N2[:,:,120:-120,120:800][ixx,-2].compute().mean())
# np.mean(N2)

2023-06-10 13:26:12,603 - distributed.worker_memory - WARNING - Worker is at 80% memory usage. Pausing worker.  Process memory: 5.06 GiB -- Worker memory limit: 6.29 GiB
2023-06-10 13:26:13,145 - distributed.worker_memory - WARNING - Worker is at 31% memory usage. Resuming worker. Process memory: 1.97 GiB -- Worker memory limit: 6.29 GiB


CPU times: user 4min 48s, sys: 1min 41s, total: 6min 29s
Wall time: 19min 41s


### $N^2_r$ (FULL)

In [13]:
rho_full=df.rho

In [14]:
z=rho_full.z_rho0.compute()

In [27]:
N2z=xr.DataArray(coords=(range(df.rho.shape[0]), range(z_interp.shape[0]),
                         range(df.rho[:,:,120:-120,120:800].shape[2]),
                         range(df.rho[:,:,120:-120,120:800].shape[3])),
            dims=['ocean_time','z_rho_psi','eta_rho','xi_rho'])

In [28]:
%%time
dz=np.diff(z_interp).mean()
for ix in range(df.rho.shape[0]):
# for ix in range(2):

    rho_interp_full=rho_full[:,:,120:-120,120:800][ix].xroms.isoslice(z_interp).compute()
    b_interp_full=xroms.buoyancy(rho_interp_full)
    N2_interp_full=np.gradient(b_interp_full,axis=0)/dz
    
    N2z[ix,:,:,:]=N2_interp_full

CPU times: user 23min 59s, sys: 7min 50s, total: 31min 49s
Wall time: 1h 21min 2s


In [29]:
N2z_mean=N2z.mean(['eta_rho','xi_rho'])

In [30]:
%%time
N2z_mean.to_netcdf('/homes/metogra/iufarias/Documents/data/new_spec/N2z_mean_full_{0}.nc'.format(month))

CPU times: user 72.4 ms, sys: 48.1 ms, total: 121 ms
Wall time: 138 ms


### $N^2_r$ (SMTH)

In [12]:
rho_smth=dt.rho

In [13]:
N2z_smth=xr.DataArray(coords=(range(dt.rho.shape[0]), range(z_interp.shape[0]),
                         range(dt.rho[:,:,120:-120,120:800].shape[2]),
                         range(dt.rho[:,:,120:-120,120:800].shape[3])),
            dims=['ocean_time','z_rho_psi','eta_rho','xi_rho'])

In [18]:
%%time
dz=np.diff(z_interp).mean()
for ix in range(dt.rho.shape[0]):
# for ix in range(2):

    rho_interp_smth=rho_smth[:,:,120:-120,120:800][ix].xroms.isoslice(z_interp).compute()
    b_interp_smth=xroms.buoyancy(rho_interp_smth)
    N2_interp_smth=np.gradient(b_interp_smth,axis=0)/dz
    
    N2z_smth[ix,:,:,:]=N2_interp_smth

CPU times: user 25min 26s, sys: 9min 55s, total: 35min 22s
Wall time: 1h 23min 14s


In [19]:
%%time
N2z_smth_mean=N2z_smth.mean(['eta_rho','xi_rho'])

CPU times: user 19.3 s, sys: 23.5 s, total: 42.8 s
Wall time: 33.5 s


In [20]:
%%time
N2z_smth_mean.to_netcdf('/homes/metogra/iufarias/Documents/data/new_spec/N2z_mean_smth_{0}.nc'.format(month))

CPU times: user 89.5 ms, sys: 52.7 ms, total: 142 ms
Wall time: 164 ms


### EKE (FULL)

In [19]:
EKE_spec_full_zt=xr.DataArray(coords=(range(df.u.shape[0]), range(z_interp.shape[0]),range(fnum)),
            dims=['ocean_time','z_rho_psi','freq_r'])

In [20]:
%%time
u_full_psi=df.u.xroms.to_grid('psi', 's_rho')[:,:,120:-120,120:800]
v_full_psi=df.v.xroms.to_grid('psi', 's_rho')[:,:,120:-120,120:800]

for ix in range(df.u.shape[0]):


    v_interp_full=v_full_psi[ix].xroms.isoslice(z_interp).compute()
    u_interp_full=u_full_psi[ix].xroms.isoslice(z_interp).compute()


    Vf_spec=xrft.isotropic_power_spectrum(v_interp_full,
                                      dim=['eta_v','xi_u'],
                              nfactor=nf,truncate='True',scaling=scl,detrend='linear', 
                                     window=wdws,window_correction=wdw_cor)
    Uf_spec=xrft.isotropic_power_spectrum(u_interp_full,
                                      dim=['eta_v','xi_u'],
                              nfactor=nf,truncate='True',scaling=scl,detrend='linear', 
                                     window=wdws,window_correction=wdw_cor)

    EKE_spec_full_zt[ix,:,:]=0.5*(Uf_spec.data + Vf_spec.data)

CPU times: user 3h 18min 43s, sys: 6h 44min 27s, total: 10h 3min 10s
Wall time: 2h 59min 33s


In [21]:
EKE_spec_full_zt['freq_r']=Vf_spec.freq_r.data
EKE_spec_full_zt['ocean_time']=df.ocean_time.data
EKE_spec_full_zt['z_rho_psi']=z_interp

In [22]:
'/homes/metogra/iufarias/Documents/data/new_spec/Eke_full_{0}.nc'.format(month)

'/homes/metogra/iufarias/Documents/data/new_spec/Eke_full_total.nc'

In [23]:
%%time
EKE_spec_full_zt.to_netcdf('/homes/metogra/iufarias/Documents/data/new_spec/Eke_full_{0}.nc'.format(month))

CPU times: user 60.9 ms, sys: 33.7 ms, total: 94.6 ms
Wall time: 494 ms


### EKE (SMTH)

In [24]:
EKE_spec_smth_zt=xr.DataArray(coords=(range(dt.u.shape[0]), range(z_interp.shape[0]),range(fnum)),
            dims=['ocean_time','z_rho_psi','freq_r'])

In [25]:
%%time
u_smth_psi=dt.u.xroms.to_grid('psi', 's_rho')[:,:,120:-120,120:800]
v_smth_psi=dt.v.xroms.to_grid('psi', 's_rho')[:,:,120:-120,120:800]

for ix in range(dt.u.shape[0]):


    v_interp_smth=v_smth_psi[ix].xroms.isoslice(z_interp).compute()
    u_interp_smth=u_smth_psi[ix].xroms.isoslice(z_interp).compute()


    Vt_spec=xrft.isotropic_power_spectrum(v_interp_smth,
                                      dim=['eta_v','xi_u'],
                              nfactor=nf,truncate='True',scaling=scl,detrend='linear', 
                                     window=wdws,window_correction=wdw_cor)
    Ut_spec=xrft.isotropic_power_spectrum(u_interp_smth,
                                      dim=['eta_v','xi_u'],
                              nfactor=nf,truncate='True',scaling=scl,detrend='linear', 
                                     window=wdws,window_correction=wdw_cor)

    EKE_spec_smth_zt[ix,:,:]=0.5*(Ut_spec.data + Vt_spec.data)

CPU times: user 3h 19min 43s, sys: 6h 46min 3s, total: 10h 5min 47s
Wall time: 2h 58min 40s


In [26]:
EKE_spec_smth_zt['freq_r']=Vt_spec.freq_r.data
EKE_spec_smth_zt['ocean_time']=dt.ocean_time.data
EKE_spec_smth_zt['z_rho_psi']=z_interp

In [27]:
%%time
EKE_spec_smth_zt.to_netcdf('/homes/metogra/iufarias/Documents/data/new_spec/Eke_smth_{0}.nc'.format(month))

CPU times: user 22.8 ms, sys: 16.6 ms, total: 39.4 ms
Wall time: 218 ms


### EPE (FULL)

In [28]:
N2r=2.175e-05

In [29]:
# EPE_spec_full_zt=xr.DataArray(coords=(range(df.rho.shape[0]), z_interp,range(fnum)),
#             dims=['ocean_time','z_rho','freq_r'])
B_spec_full_zt=xr.DataArray(coords=(range(df.rho.shape[0]), z_interp,range(fnum)),
            dims=['ocean_time','z_rho','freq_r'])

In [30]:
%%time
rho_full_psi=df.rho[:,:,120:-120,120:800]


for iy in range(df.rho.shape[0]):
    rho_interp_full=rho_full_psi[iy].xroms.isoslice(z_interp).compute()
    b_interp_full=xroms.buoyancy(rho_interp_full)


    Bf_spec=xrft.isotropic_power_spectrum(b_interp_full.drop(['nav_lat_rho', 'nav_lon_rho']),
                                      dim=['eta_rho','xi_rho'],
                              nfactor=nf,truncate='True',scaling=scl,detrend='linear', 
                                     window=wdws,window_correction=wdw_cor)

    B_spec_full_zt[iy,:,:]=(Bf_spec.data)

CPU times: user 1h 51min 50s, sys: 4h 9min 23s, total: 6h 1min 13s
Wall time: 1h 40min 25s


In [31]:
# EPE_spec_full_zt['freq_r']=Bf_spec.freq_r.data
# EPE_spec_full_zt['ocean_time']=df.ocean_time.data
# EPE_spec_full_zt['z_rho']=z_interp

B_spec_full_zt['freq_r']=Bf_spec.freq_r.data
B_spec_full_zt['ocean_time']=df.ocean_time.data
B_spec_full_zt['z_rho']=z_interp

In [32]:
# EPE_spec_full_zt.to_netcdf('/homes/metogra/iufarias/Documents/data/new_spec/Epe_full_{0}.nc'.format(month))
B_spec_full_zt.to_netcdf('/homes/metogra/iufarias/Documents/data/new_spec/B_full_{0}.nc'.format(month))

### EPE(SMTH)

In [33]:
# EPE_spec_smth_zt=xr.DataArray(coords=(range(dt.rho.shape[0]), z_interp,range(fnum)),
#             dims=['ocean_time','z_rho','freq_r'])
B_spec_smth_zt=xr.DataArray(coords=(range(dt.rho.shape[0]), z_interp,range(fnum)),
            dims=['ocean_time','z_rho','freq_r'])

In [34]:
%%time
rho_smth_psi=dt.rho[:,:,120:-120,120:800]


for iy in range(dt.rho.shape[0]):
    rho_interp_smth=rho_smth_psi[iy].xroms.isoslice(z_interp).compute()
    b_interp_smth=xroms.buoyancy(rho_interp_smth)


    Bt_spec=xrft.isotropic_power_spectrum(b_interp_smth.drop(['nav_lat_rho', 'nav_lon_rho']),
                                      dim=['eta_rho','xi_rho'],
                              nfactor=nf,truncate='True',scaling=scl,detrend='linear', 
                                     window=wdws,window_correction=wdw_cor)

    B_spec_smth_zt[iy,:,:]=(Bt_spec.data)

CPU times: user 1h 54min 2s, sys: 4h 18min 24s, total: 6h 12min 26s
Wall time: 1h 41min 44s


In [35]:
# EPE_spec_smth_zt['freq_r']=Bt_spec.freq_r.data
# EPE_spec_smth_zt['ocean_time']=dt.ocean_time.data
# EPE_spec_smth_zt['z_rho']=z_interp

B_spec_smth_zt['freq_r']=Bt_spec.freq_r.data
B_spec_smth_zt['ocean_time']=dt.ocean_time.data
B_spec_smth_zt['z_rho']=z_interp

In [36]:
# EPE_spec_smth_zt.to_netcdf('/homes/metogra/iufarias/Documents/data/new_spec/Epe_smth_{0}.nc'.format(month))
B_spec_smth_zt.to_netcdf('/homes/metogra/iufarias/Documents/data/new_spec/Epe_smth_{0}.nc'.format(month))

### w'b' (FULL)

In [29]:
wb_spec_full_zt=xr.DataArray(coords=(range(df.rho.shape[0]), z_interp,range(fnum)),
            dims=['ocean_time','z_rho','freq_r'])

In [30]:
%%time
w_full_psi=df.w[:,:,120:-120,120:800]
rho_full_psi=df.rho[:,:,120:-120,120:800]

for iy in range(df.rho.shape[0]):
    rho_interp_full=rho_full_psi[iy].xroms.isoslice(z_interp).compute()
    b_interp_full=xroms.buoyancy(rho_interp_full)
    
    w_interp_full=w_full_psi[iy].xroms.isoslice(z_interp).compute()


    WBf_spec=xrft.isotropic_cross_spectrum(w_interp_full.drop(['nav_lat_rho', 'nav_lon_rho']),
                                           b_interp_full.drop(['nav_lat_rho', 'nav_lon_rho']),
                                      dim=['eta_rho','xi_rho'],
                              nfactor=nf,truncate='True',scaling=scl,detrend='linear', 
                                     window=wdws,window_correction=wdw_cor).real

    wb_spec_full_zt[iy,:,:]=(WBf_spec.data)

CPU times: user 27min 8s, sys: 1h 4min 54s, total: 1h 32min 3s
Wall time: 16min 18s


In [31]:
wb_spec_full_zt['freq_r']=WBf_spec.freq_r.data
wb_spec_full_zt['ocean_time']=df.ocean_time.data
wb_spec_full_zt['z_rho']=z_interp

In [32]:
wb_spec_full_zt.to_netcdf('/homes/metogra/iufarias/Documents/data/new_spec/wb_full_{0}.nc'.format(month))

### w'b' (SMTH)

In [33]:
wb_spec_smth_zt=xr.DataArray(coords=(range(dt.rho.shape[0]), z_interp,range(fnum)),
            dims=['ocean_time','z_rho','freq_r'])

In [34]:
%%time
w_smth_psi=dt.w[:,:,120:-120,120:800]
rho_smth_psi=dt.rho[:,:,120:-120,120:800]

for iy in range(dt.rho.shape[0]):
    rho_interp_smth=rho_smth_psi[iy].xroms.isoslice(z_interp).compute()
    b_interp_smth=xroms.buoyancy(rho_interp_smth)
    
    w_interp_smth=w_smth_psi[iy].xroms.isoslice(z_interp).compute()


    WBt_spec=xrft.isotropic_cross_spectrum(w_interp_smth.drop(['nav_lat_rho', 'nav_lon_rho']),
                                           b_interp_smth.drop(['nav_lat_rho', 'nav_lon_rho']),
                                      dim=['eta_rho','xi_rho'],
                              nfactor=nf,truncate='True',scaling=scl,detrend='linear', 
                                     window=wdws,window_correction=wdw_cor).real

    wb_spec_smth_zt[iy,:,:]=(WBt_spec.data)

CPU times: user 27min 20s, sys: 1h 7min 28s, total: 1h 34min 49s
Wall time: 17min


In [35]:
wb_spec_smth_zt['freq_r']=WBt_spec.freq_r.data
wb_spec_smth_zt['ocean_time']=dt.ocean_time.data
wb_spec_smth_zt['z_rho']=z_interp

In [36]:
wb_spec_smth_zt.to_netcdf('/homes/metogra/iufarias/Documents/data/new_spec/wb_smth_{0}.nc'.format(month))

2023-06-11 10:15:10,022 - tornado.application - ERROR - Uncaught exception GET /status/ws (::1)
HTTPServerRequest(protocol='http', host='127.0.0.1:8787', method='GET', uri='/status/ws', version='HTTP/1.1', remote_ip='::1')
Traceback (most recent call last):
  File "/homes/metogra/iufarias/miniconda3/envs/XROMS/lib/python3.8/site-packages/tornado/websocket.py", line 954, in _accept_connection
    open_result = handler.open(*handler.open_args, **handler.open_kwargs)
  File "/homes/metogra/iufarias/miniconda3/envs/XROMS/lib/python3.8/site-packages/tornado/web.py", line 3173, in wrapper
    return method(self, *args, **kwargs)
  File "/homes/metogra/iufarias/miniconda3/envs/XROMS/lib/python3.8/site-packages/bokeh/server/views/ws.py", line 149, in open
    raise ProtocolError("Token is expired.")
bokeh.protocol.exceptions.ProtocolError: Token is expired.


In [251]:
# fig,ax=plt.subplots(2,1,figsize=(9,6))


# im1=ax[0].pcolormesh(AA.freq_r/dx,AA.z_rho_psi,AA.mean(axis=0)/dx*1e3,
#                vmin=0,vmax=6e-5,cmap='inferno')
# ax[0].set_xscale('log')
# ax[0].set_xlim(1e-2,0)
# ax[0].set_title(r'(a)',color='k',fontsize=20)
# # ax[0].set_ylabel('Depth [m]',fontsize=20,x=0.1)



# im2=ax[1].pcolormesh(BB.freq_r/dx,BB.z_rho_psi,BB.mean(axis=0)/dx*1e3,
#                vmin=0,vmax=6e-5,cmap='inferno')
# ax[1].set_xscale('log')
# ax[1].set_xlim(1e-2,0)
# ax[1].set_title(r'(b)',fontsize=20)
# # ax[1].set_ylabel('Depth [m]',fontsize=20,x=0.1)
# fig.supylabel('Depth [m]',fontsize=20)
# fig.supxlabel(r'Wavenumber [km$^{-1}$]',fontweight='bold',fontsize=20,y=-0.00001)
    

    
# fig.subplots_adjust(right=0.8,hspace=0.4)
# cbar_ax = fig.add_axes([0.85, 0.15, 0.05, 0.7])
# cbar=fig.colorbar(im, cax=cbar_ax)
# # cbar=fig.colorbar(im,cax=ax[0]
# cbar.set_label(r'Spectral Density [m$^3\,$s$^{-3}$]',labelpad=12)

# # plt.tight_layout()
# plt.savefig('/homes/metogra/iufarias/Documents/figures/2_CROCO/CROCO_zintegrated/z_int/wb_z.png')